In [180]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib 
import pickle

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn import metrics

In [181]:
df = pd.read_csv("Churn_Modelling.csv")

df_x = df.iloc[:, 3:13]
df_y = df.iloc[:, 13]

In [182]:
def clean_data(df):

    le = LabelEncoder()
    df.Gender = le.fit_transform(df.Gender)
    df = pd.get_dummies(data = df, columns=["Geography"], drop_first = False)
    df = df.sort_index(axis=1)
    return df

In [183]:
df_x = clean_data(df_x)

In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 0)
pickle.dump(df_x.columns, open("columns.pkl", 'wb'))

In [185]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
pickle.dump(scaler, open("std_scaler.pkl", 'wb'))
print(X_test)
print(X_train.shape[1])

[[-0.36890377  0.8793029  -0.55204276 ...  0.9687384  -0.92159124
   1.04473698]
 [ 0.10961719  0.42972196 -1.31490297 ... -1.03227043 -0.92159124
  -1.031415  ]
 [ 0.30102557  0.30858264  0.57162971 ...  0.9687384  -0.92159124
   1.04473698]
 ...
 [-0.27319958  1.29745526 -0.74791227 ... -1.03227043  0.8095029
  -1.37744033]
 [-0.46460796  1.05975239 -0.00566991 ...  0.9687384  -0.92159124
  -0.33936434]
 [-0.84742473  0.82026342 -0.79945688 ... -1.03227043 -0.92159124
   1.04473698]]
12


In [188]:
import xgboost as xgb
model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200,learning_rate=0.16)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.16, max_delta_step=0,
              max_depth=50, min_child_weight=1, missing=None, n_estimators=200,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [191]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
predictions = model.predict_proba(X_test)
#print ("\naccuracy_score :",accuracy_score(y_test,predictions))

In [192]:
predictions

array([[0.91999733, 0.08000269],
       [0.95326316, 0.04673682],
       [0.9971275 , 0.00287252],
       ...,
       [0.99371326, 0.00628672],
       [0.89380795, 0.10619205],
       [0.9683398 , 0.03166021]], dtype=float32)

In [193]:
# save the model so created above into a picle.
pickle.dump(model, open('model.pkl', 'wb')) 